In [49]:
from importlib import reload
from dotenv import load_dotenv

load_dotenv(r"C:\Users\vireima\tie-dashboard\.env")

import os
#print(os.getenv("SEVERA_CLIENT_SCOPE"))

import pandas as pd

import src.severa.base_client
import src.severa.fetch
from src.daterange import DateRange
reload(src.severa.base_client)
reload(src.severa.fetch)

span = DateRange(540)

#async with src.severa.fetch.Fetcher() as f:
#    df = await f.get_resource_allocations(span)

In [50]:
async with src.severa.fetch.Fetcher() as f:
    df2 = await f.get_allocations_with_maxes(span)

df2.head()

2023-05-31 23:30:35.872 | SUCCESS  | src.severa.base_client:get_with_retries:123 - HTTP/1.1 GET users [0]: Response (9) in 0.45s.
2023-05-31 23:30:36.147 | SUCCESS  | src.severa.base_client:get_with_retries:123 - HTTP/1.1 GET activities [0]: Response (0) in 0.25s.
2023-05-31 23:30:36.291 | SUCCESS  | src.severa.base_client:get_with_retries:123 - HTTP/1.1 GET activities [0]: Response (3) in 0.39s.
2023-05-31 23:30:36.363 | SUCCESS  | src.severa.base_client:get_with_retries:123 - HTTP/1.1 GET activities [0]: Response (20) in 0.45s.
2023-05-31 23:30:36.548 | SUCCESS  | src.severa.base_client:get_with_retries:123 - HTTP/1.1 GET activities [0]: Response (1) in 0.23s.
2023-05-31 23:30:36.558 | SUCCESS  | src.severa.base_client:get_with_retries:123 - HTTP/1.1 GET activities [0]: Response (0) in 0.66s.
2023-05-31 23:30:36.567 | SUCCESS  | src.severa.base_client:get_with_retries:123 - HTTP/1.1 GET activities [0]: Response (0) in 0.66s.
2023-05-31 23:30:36.797 | SUCCESS  | src.severa.base_client

,businessunit-user,value,user,project,phase,date,id,forecast-date,type
0,TIE,0.4,78ddf0fe-9e39-5e5e-5096-a80409925de9,9a6e0602-cd0b-b8f2-14d3-2690c3db82f1,38056486-4604-35f9-c092-c333f37358d4,2023-05-31 00:00:00+00:00,allocation,2023-08-02 00:00:00+00:00,external
1,TIE,0.4,78ddf0fe-9e39-5e5e-5096-a80409925de9,9a6e0602-cd0b-b8f2-14d3-2690c3db82f1,38056486-4604-35f9-c092-c333f37358d4,2023-05-31 00:00:00+00:00,allocation,2023-08-03 00:00:00+00:00,external
2,TIE,0.4,78ddf0fe-9e39-5e5e-5096-a80409925de9,9a6e0602-cd0b-b8f2-14d3-2690c3db82f1,38056486-4604-35f9-c092-c333f37358d4,2023-05-31 00:00:00+00:00,allocation,2023-08-04 00:00:00+00:00,external
3,TIE,0.4,78ddf0fe-9e39-5e5e-5096-a80409925de9,9a6e0602-cd0b-b8f2-14d3-2690c3db82f1,38056486-4604-35f9-c092-c333f37358d4,2023-05-31 00:00:00+00:00,allocation,2023-08-05 00:00:00+00:00,external
4,TIE,0.4,78ddf0fe-9e39-5e5e-5096-a80409925de9,9a6e0602-cd0b-b8f2-14d3-2690c3db82f1,38056486-4604-35f9-c092-c333f37358d4,2023-05-31 00:00:00+00:00,allocation,2023-08-06 00:00:00+00:00,external


In [61]:
df2[df2["date"] == df2["date"].max()]

,businessunit-user,value,user,project,phase,date,id,forecast-date,type
0,TIE,0.4,78ddf0fe-9e39-5e5e-5096-a80409925de9,9a6e0602-cd0b-b8f2-14d3-2690c3db82f1,38056486-4604-35f9-c092-c333f37358d4,2023-05-31 00:00:00+00:00,allocation,2023-08-02 00:00:00+00:00,external
1,TIE,0.4,78ddf0fe-9e39-5e5e-5096-a80409925de9,9a6e0602-cd0b-b8f2-14d3-2690c3db82f1,38056486-4604-35f9-c092-c333f37358d4,2023-05-31 00:00:00+00:00,allocation,2023-08-03 00:00:00+00:00,external
2,TIE,0.4,78ddf0fe-9e39-5e5e-5096-a80409925de9,9a6e0602-cd0b-b8f2-14d3-2690c3db82f1,38056486-4604-35f9-c092-c333f37358d4,2023-05-31 00:00:00+00:00,allocation,2023-08-04 00:00:00+00:00,external
3,TIE,0.4,78ddf0fe-9e39-5e5e-5096-a80409925de9,9a6e0602-cd0b-b8f2-14d3-2690c3db82f1,38056486-4604-35f9-c092-c333f37358d4,2023-05-31 00:00:00+00:00,allocation,2023-08-05 00:00:00+00:00,external
4,TIE,0.4,78ddf0fe-9e39-5e5e-5096-a80409925de9,9a6e0602-cd0b-b8f2-14d3-2690c3db82f1,38056486-4604-35f9-c092-c333f37358d4,2023-05-31 00:00:00+00:00,allocation,2023-08-06 00:00:00+00:00,external
...,...,...,...,...,...,...,...,...,...
28211,TIE,0.0,2441bfbf-78f2-0173-b0a1-8634ce167a0d,<NA>,<NA>,2023-05-31 00:00:00+00:00,allocation,2024-11-17 00:00:00+00:00,max
28212,TIE,7.5,2441bfbf-78f2-0173-b0a1-8634ce167a0d,<NA>,<NA>,2023-05-31 00:00:00+00:00,allocation,2024-11-18 00:00:00+00:00,max
28213,TIE,7.5,2441bfbf-78f2-0173-b0a1-8634ce167a0d,<NA>,<NA>,2023-05-31 00:00:00+00:00,allocation,2024-11-19 00:00:00+00:00,max
28214,TIE,7.5,2441bfbf-78f2-0173-b0a1-8634ce167a0d,<NA>,<NA>,2023-05-31 00:00:00+00:00,allocation,2024-11-20 00:00:00+00:00,max


In [21]:
from datetime import timedelta
delta = timedelta(days=30)
grouped = (
        df2[df2["forecast-date"].between(df2["date"], df2["date"] + delta)]
        .groupby(["date", "type"])["value"]
        .sum()
        .reset_index()
    )
grouped

,date,type,value
0,2023-05-31 00:00:00+00:00,external,626.257431
1,2023-05-31 00:00:00+00:00,internal,169.568058
2,2023-05-31 00:00:00+00:00,max,1330.5


In [36]:
g = grouped.pivot(columns="type", values="value", index="date").reset_index()
g

type,date,external,internal,max
0,2023-05-31 00:00:00+00:00,626.257431,169.568058,1330.5


In [43]:
g["total"] = g["external"] + g["internal"]
g["billing-rate"] = g["external"] / g["total"]
g["allocation-rate"] = g["total"] / g["max"]
g

type,date,external,internal,max,total,billing-rate,allocation-rate
0,2023-05-31 00:00:00+00:00,626.257431,169.568058,1330.5,795.825489,0.786928,0.59814


In [42]:
g.melt(id_vars=["date"])

,date,type,value
0,2023-05-31 00:00:00+00:00,external,626.257431
1,2023-05-31 00:00:00+00:00,internal,169.568058
2,2023-05-31 00:00:00+00:00,max,1330.5
3,2023-05-31 00:00:00+00:00,total,795.825489
4,2023-05-31 00:00:00+00:00,billing-rate,0.786928
5,2023-05-31 00:00:00+00:00,allocation-rate,0.59814


In [30]:
from dotenv import load_dotenv

load_dotenv(r"C:\Users\vireima\tie-dashboard\.env")

import src.database
reload(src.database)

b = src.database.Base("kpi-dev", "allocations")

In [31]:
x = b.find()

2023-05-31 09:01:45.483 | INFO     | src.database:find:32 - Query '{}' resulted in 23438 results in 12.39s.


In [32]:
x.loc[:,["forecast-date","value"]]

,forecast-date,value
0,2023-02-27,0.642713
1,2023-02-28,0.642713
2,2023-03-01,0.642713
3,2023-03-02,0.642713
4,2023-03-03,0.642713
...,...,...
23433,2023-06-26,0.333333
23434,2023-06-27,0.333333
23435,2023-06-28,0.333333
23436,2023-06-29,0.333333


In [227]:
# Quick mock data
a = pd.Series(1, index=pd.date_range("2023-01-01", "2023-01-05", freq="D"))
b = pd.Series(1, index=pd.date_range("2023-01-03", "2023-01-08", freq="D"))
c = pd.Series(1, index=pd.date_range("2023-01-04", "2023-01-12", freq="D"))

df = pd.DataFrame({"user": ["A", "B", "B"], "unit": ["TIE"]*3, "values":[a,b,c]})

# Add series with datetime indices together
def combine_series(df):
    return reduce(partial(pd.Series.add,fill_value=0), df)

# This works nicely...
#ok = combine_series(df["values"])

# ...outputs a DF, indices are OK:
# 2023-01-01    1.0
# 2023-01-02    1.0
# 2023-01-03    2.0
# 2023-01-04    3.0
# ...
# 2023-01-12    1.0
# Freq: D, dtype: float64

# This 'works' in that the values are summed correctly,
# but the indices are gone now
#not_ok = df.groupby(["unit", "user"], as_index=False).agg(combine_series)

# This is seemingly a np array
#not_ok.loc["B", "values"]
# array([1., 2., 2., 2., 2., 2., 1., 1., 1., 1.])

grouping = ["unit", "user"]
grouped = df.groupby(grouping)

pd.DataFrame({"values":combine_series(v["values"])} | dict(zip(grouping, k)) for k, v in grouped)



,values,unit,user
0,2023-01-01 1 2023-01-02 1 2023-01-03 ...,TIE,A
1,2023-01-03 1.0 2023-01-04 2.0 2023-01-05...,TIE,B


In [298]:


hours_per_day = 1.5
index=pd.date_range(
    "2023-05-01",
    "2023-07-30",
    freq="D",
)
pd.DataFrame(index=index, data={"date":"2023-05-20", "user": "ville","value":hours_per_day})[]

,date,user,value
2023-05-01,2023-05-20,ville,1.5
2023-05-02,2023-05-20,ville,1.5
2023-05-03,2023-05-20,ville,1.5
2023-05-04,2023-05-20,ville,1.5
2023-05-05,2023-05-20,ville,1.5
...,...,...,...
2023-07-26,2023-05-20,ville,1.5
2023-07-27,2023-05-20,ville,1.5
2023-07-28,2023-05-20,ville,1.5
2023-07-29,2023-05-20,ville,1.5
